In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
from tqdm import tqdm

In [5]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [6]:
df= pd.read_csv('/content/drive/MyDrive/MiSongGiny/data/test_data/track_2_public_test_rev002.csv', encoding='utf-8')

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "no0ne-97/misoginia-roberta-base-bne-V3"

# Carga el modelo y el tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# A GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [30]:
new_df = pd.DataFrame(columns=['id', 'label'])

In [31]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probs, dim=1).item()

    return predicted_class, probs.squeeze().tolist()

In [32]:
for index, row, in tqdm(df.iterrows()):
    id = row["id"]
    lyrics = row["lyrics"]

    if len(lyrics) > 1000:
        lyrics = lyrics[:1000]
    label, probabilidades = predict(lyrics)

    df_new_row = pd.DataFrame({'id': [id], 'label': [label]})
    df_new_row['label'] = df_new_row['label'].map({0: 'NM', 1: 'M'})
    new_df = pd.concat([new_df, df_new_row], ignore_index=True)

293it [00:10, 28.53it/s]


In [33]:
new_df['label'].value_counts()

,count
label,
NR,156
S,137


In [28]:
new_df.to_csv('/content/drive/MyDrive/MiSongGiny/answers/track_1_predictions_roberta.csv', index=False)

In [37]:
# prompt: Open the dataframe in /content/drive/MyDrive/MiSongGiny/answers/track_1_predictions_roberta.csv and convert the label NR in NM and the label S into M and save again

import pandas as pd

# Load the dataframe
df = pd.read_csv('/content/drive/MyDrive/MiSongGiny/answers/track_1_predictions_beto.csv')

# Replace labels
df['label'] = df['label'].replace({'NR': 'NM', 'S': 'M'})

# Save the modified dataframe
df.to_csv('/content/drive/MyDrive/MiSongGiny/answers/track_1_predictions_beto.csv', index=False)
